import

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

데이터 준비

In [3]:
# 이미지 크기
input_size = (224, 224, 3)
EPOCHS = 100
batch_size = 64
img_height = 224
img_width = 224
data_dir = "./data_cropped_alexnet"

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 46931 files belonging to 16 classes.
Using 37545 files for training.
Found 46931 files belonging to 16 classes.
Using 9386 files for validation.


## 모델 정의

In [5]:
class VGG19(Model):
    def __init__(self):
        super(VGG19, self).__init__()

        # Block 1
        self.block1conv1 = Conv2D(64, (3, 3), activation='relu', padding="same", input_shape=input_size)
        self.block1conv2 = Conv2D(64, (3, 3), activation='relu', padding="same")
        self.block1maxpool = MaxPooling2D((2, 2), strides=(2, 2))

        # Block 2
        self.block2conv1 = Conv2D(128, (3, 3), activation='relu', padding='same')
        self.block2conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')
        self.block2maxpool = MaxPooling2D((2, 2), strides=(2, 2))

        # Block 3
        self.block3conv1 = Conv2D(256, (3, 3), activation='relu', padding='same')
        self.block3conv2 = Conv2D(256, (3, 3), activation='relu', padding='same')
        self.block3conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')
        self.block3maxpool = MaxPooling2D((2, 2), strides=(2, 2))

        # Block 4
        self.block4conv1 = Conv2D(512, (3, 3), activation='relu', padding='same')
        self.block4conv2 = Conv2D(512, (3, 3), activation='relu', padding='same')
        self.block4conv3 = Conv2D(512, (3, 3), activation='relu', padding='same')
        self.block4maxpool = MaxPooling2D((2, 2), strides=(2, 2))

        # Block 5
        self.block5conv1 = Conv2D(512, (3, 3), activation='relu', padding='same')
        self.block5conv2 = Conv2D(512, (3, 3), activation='relu', padding='same')
        self.block5conv3 = Conv2D(512, (3, 3), activation='relu', padding='same')
        self.block5maxpool = MaxPooling2D((2, 2), strides=(2, 2))

        # FC
        self.flatten = Flatten()
        self.fc1 = Dense(4096, activation='relu')
        self.fc2 = Dense(4096, activation='relu')
        self.fc3 = Dense(16, activation='softmax')


    def call(self, x):
        # Block 1
        x = self.block1conv1(x)
        x = self.block1conv2(x)
        x = self.block1maxpool(x)

        # Block 2
        x = self.block2conv1(x)
        x = self.block2conv2(x)
        x = self.block2maxpool(x)

        # Block 3
        x = self.block3conv1(x)
        x = self.block3conv2(x)
        x = self.block3conv3(x)
        x = self.block3maxpool(x)

        # Block 4
        x = self.block4conv1(x)
        x = self.block4conv2(x)
        x = self.block4conv3(x)
        x = self.block4maxpool(x)

        # Block 5
        x = self.block5conv1(x)
        x = self.block5conv2(x)
        x = self.block5conv3(x)
        x = self.block5maxpool(x)

        # FC
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

model = VGG19()

손실함수와 최적화함수 정의

In [6]:
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(lr=0.001)

# 모델의 손실과 성능을 측정할 지표 선택.
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

C:\anaconda3\envs\tf-gpu2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


모델 훈련함수 정의

In [7]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_obj(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

모델 테스트함수 정의

In [8]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_obj(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

모델 구동

In [9]:
f = open("./VGG19_results.txt", 'w', encoding='utf-8')
for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print(template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))
  f.write(template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100)+'\n')
f.close()

에포크: 1, 손실: 21.23874282836914, 정확도: 82.562255859375, 테스트 손실: 0.564388632774353, 테스트 정확도: 84.70062255859375
에포크: 2, 손실: 10.838385581970215, 정확도: 84.77693176269531, 테스트 손실: 0.44342124462127686, 테스트 정확도: 87.1777114868164
에포크: 3, 손실: 7.308974266052246, 정확도: 87.15940856933594, 테스트 손실: 0.36251503229141235, 테스트 정확도: 89.1434097290039
에포크: 4, 손실: 5.527816295623779, 정확도: 88.81342315673828, 테스트 손실: 0.31455060839653015, 테스트 정확도: 90.45919036865234
에포크: 5, 손실: 4.451180934906006, 정확도: 90.02264404296875, 테스트 손실: 0.28769803047180176, 테스트 정확도: 91.26358795166016
에포크: 6, 손실: 3.7305829524993896, 정확도: 90.92288970947266, 테스트 손실: 0.26667192578315735, 테스트 정확도: 91.80339813232422
에포크: 7, 손실: 3.2145705223083496, 정확도: 91.60511779785156, 테스트 손실: 0.2506030201911926, 테스트 정확도: 92.30769348144531
에포크: 8, 손실: 2.8264334201812744, 정확도: 92.15141296386719, 테스트 손실: 0.23560361564159393, 테스트 정확도: 92.71654510498047
에포크: 9, 손실: 2.52374529838562, 정확도: 92.61685943603516, 테스트 손실: 0.22664406895637512, 테스트 정확도: 92.95404815673828
에포크: 